In [5]:
import urllib.request

manifold_geom_url = 'https://tinyurl.com/rp7grox'
manifold_mesh_url = 'https://tinyurl.com/wojemuh'

def download(url, filename='dagmc.h5m'):
    """
    Helper function for retrieving dagmc models
    """
    u = urllib.request.urlopen(url)
    
    if u.status != 200:
        raise RuntimeError("Failed to download file.")
    
    # save file as dagmc.h5m
    with open(filename, 'wb') as f:
        f.write(u.read())

# Unstructured Mesh Tallies with CAD Geometry in OpenMC

In the first notebook on this topic, we looked at how to set up a tally using an unstructured mesh in OpenMC.
In this notebook, we will explore using unstructured mesh in conjunction with CAD-based geometry to perform detailed geometry analysis on complex geomerty.

_NOTE: This notebook will not run successfully if OpenMC has not been built with DAGMC support enabled._

In [8]:
import openmc
import openmc.lib

assert(openmc.lib._dagmc_enabled())

The model we'll be looking at today is a steel piping manifold:
![CAD Manifold](./images/manifold-cad.png)


This is a nice example of a model which would be extremely difficult to model using CSG. To get started, we'll need two files: 
  1. the DAGMC gometry file on which we'll track particles and 
  2. a tetrahedral mesh of the piping structure on which we'll score tallies
  
To start, let's create the materials we'll need for this problem. The pipes are steel and we'll model the surrounding area as air.

In [17]:
air = openmc.Material(name='air')
air.set_density('g/cc', 0.001205)
air.add_nuclide('N14',0.781557629247)
air.add_nuclide('N15',0.002873370753)
air.add_nuclide('O16',0.210668126508)
air.add_nuclide('O17',7.9873492e-05)
air.add_nuclide('Ar36',1.53456e-05)
air.add_nuclide('Ar38',2.8934e-06)
air.add_nuclide('Ar40',0.004581761)

steel = openmc.Material(name='steel')
steel.set_density('g/cc', 8.0)
steel.add_nuclide('Si28',0.0092672382464)
steel.add_nuclide('Si29',0.00047056391679999997)
steel.add_nuclide('Si30',0.00031019783679999996)
steel.add_nuclide('P31',0.00023)
steel.add_nuclide('S32',0.000218593702)
steel.add_nuclide('S33',1.721987e-06)
steel.add_nuclide('S34',9.650777000000001e-06)
steel.add_nuclide('S36',3.3534e-08)
steel.add_nuclide('Mn55',0.011014)
steel.add_nuclide('Fe54',0.03910305)
steel.add_nuclide('Fe56',0.6138342600000001)
steel.add_nuclide('Fe57',0.01417611)
steel.add_nuclide('Fe58',0.0018865800000000001)
steel.add_nuclide('Ni58',0.08169227999999999)
steel.add_nuclide('Ni60',0.03146772)
steel.add_nuclide('Ni61',0.00136788)
steel.add_nuclide('Ni62',0.0043614000000000005)
steel.add_nuclide('Ni64',0.00111072)
steel.add_nuclide('Mo100',0.0024360000000000002)
steel.add_nuclide('Mo92',0.0036622500000000006)
steel.add_nuclide('Mo94',0.0022967499999999997)
steel.add_nuclide('Mo95',0.00396825)
steel.add_nuclide('Mo96',0.00416825)
steel.add_nuclide('Mo97',0.0023955)
steel.add_nuclide('Mo98',0.006073)

materials = openmc.Materials([air, steel])
materials.export_to_xml()

In [10]:
download(manifold_geom_url)
download(manifold_mesh_url, 'manifold.h5m')

Next we'll create a point source at the entrance the single pipe on the low side of the model.

In [13]:
src_pnt = openmc.stats.Point(xyz=(0.0, 0.0, 0.0))
src_energy = openmc.stats.Discrete(x=[10.0], p=[1.0])

source = openmc.Source(space=src_pnt, energy=src_energy)

settings = openmc.Settings()
settings.source = source

settings.run_mode = "fixed source"
settings.batches = 10
settings.particles = 5000

And we'll indicate that we're using a CAD-based geometry.

In [15]:
settings.dagmc = True

settings.export_to_xml()

We'll run a few particles through this geometry to make sure everything is working properly.

In [20]:
openmc.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

Now let's setup the unstructured mesh tally. We'll do this the same way we did in the previous notebook.

In [25]:
unstructured_mesh = openmc.UnstructuredMesh(filename="manifold.h5m")

mesh_filter = openmc.MeshFilter(unstructured_mesh)

tally = openmc.Tally()
tally.filters = [mesh_filter]
tally.scores = ['flux']
tally.estimator = 'tracklength'


tallies = openmc.Tallies([tally])
tallies.export_to_xml()

In [29]:
settings.batches = 200
settings.export_to_xml()

In [30]:
openmc.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

 Simulating batch 174
 Simulating batch 175
 Simulating batch 176
 Simulating batch 177
 Simulating batch 178
 Simulating batch 179
 Simulating batch 180
 Simulating batch 181
 Simulating batch 182
 Simulating batch 183
 Simulating batch 184
 Simulating batch 185
 Simulating batch 186
 Simulating batch 187
 Simulating batch 188
 Simulating batch 189
 Simulating batch 190
 Simulating batch 191
 Simulating batch 192
 Simulating batch 193
 Simulating batch 194
 Simulating batch 195
 Simulating batch 196
 Simulating batch 197
 Simulating batch 198
 Simulating batch 199
 Simulating batch 200
 Creating state point statepoint.200.h5...
 Writing unstructured mesh tally_3.200.vtk...

 =======================>     TIMING STATISTICS     <=======================

 Total time for initialization     = 3.6126e+01 seconds
   Reading cross sections          = 1.5844e+00 seconds
 Total time in simulation          = 2.5434e+02 seconds
   Time in transport only          = 2.0793e+02 seconds
   Time in act

Again we should see that `tally_1.100.vtk` file which we can use to visualize our results in VisIt or another tool of your choice that supports VTK files.

In [31]:
!ls *.vtk

tally_1.100.vtk  tally_3.100.vtk  tally_3.200.vtk


![](./images/manifold_flux.png)

S of the elements have no score We indeed see that the flux values are larger near the source at the bottom of the model